In [ ]:
# In colab please uncomment this to install Atari
# Box2d is a 2D physics engine.
!pip install box2d-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.5/374.5 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for box2d-py
  Running setup.py clean for box2d-py
Failed to build box2d-py
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (box2d-py)


In [ ]:
# And for visualization on Colab install
!pip install pyglet
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 962.0/962.0 kB 11.6 MB/s eta 0:00:00


In [ ]:
import gym

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from IPython import display as ipythondisplay  ## Needed on colab

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
## Need to set a virtual Display on colab, else ipythondisplay would not work
from pyvirtualdisplay import Display
display = Display(visible=0, size=(400, 300))
display.start()

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


FileNotFoundError: [Errno 2] No such file or directory: 'Xvfb'

## Physics Engine

In [ ]:
env = gym.make('LunarLander-v2')
obs = env.reset()
img = env.render(mode='rgb_array')
env.close()
plt.imshow(img)
#ipythondisplay.clear_output(wait=True)  #Colab

# Classic Control

In [ ]:
env = gym.make('CartPole-v0')
env.reset()
img = env.render(mode='rgb_array')
env.close()
plt.imshow(img)

## Atari Games

In [ ]:
!wget http://www.atarimania.com/roms/Roms.rar
!mkdir /content/ROM/
!unrar e /content/Roms.rar /content/ROM/
!python -m atari_py.import_roms /content/ROM/

In [ ]:
env = gym.make('SpaceInvaders-v0')
env.reset()
img = env.render(mode='rgb_array')
env.close()
plt.imshow(img)

In [ ]:
from gym import envs

envall = envs.registry.all()
len(envall)

In [ ]:
from tqdm import tqdm
List = []
for e in tqdm(envall):
    try:
        env = e.make()
        List.append([e.id, env.observation_space, env.action_space, env.reward_range])
        env.close()
    except:
        continue



In [ ]:
df = pd.DataFrame(List, columns= ['Environment', 'Observation Space', 'Action Space', 'Reward'])
df.sample(20)

In [ ]:
env = gym.make('MountainCar-v0')
print(f"The Observation space is        {env.observation_space}" )

print(f"Upper Bound for Env Observation {env.observation_space.high}")
print(f"Lower Bound for Env Observation {env.observation_space.low}")
print(f"Action Space                    {env.action_space}")

env.seed(0)
obs = env.reset()
print(f"The initial observation is      {obs}")
# # Take a random actionget the new observation space
new_obs, reward, done, info = env.step(env.action_space.sample())
print(f"The new observation is          {new_obs}")
env.close()

# Working with Wrappers

In [ ]:
class ClippedRewards(gym.RewardWrapper):
    def __init__(self, env):
        gym.RewardWrapper.__init__(self, env)
        self.reward_range = (-10,10)

    def reward(self, reward):
        """Clip to {+10, 0, -10} by its sign."""
        return reward if reward >= -10 and reward <= 10 else 10 * np.sign(reward)

In [ ]:
env = gym.make("CartPole-v0")
print("Original Rewards:", env.reward_range)
env.close()
env = ClippedRewards(gym.make("CartPole-v0"))
print(f'Clipped reward range: {env.reward_range}')
env.close()


# Working with Wrappers

In [ ]:
from collections import deque
from gym import spaces
import numpy as np

class ConcatObservations(gym.Wrapper):
    def __init__(self, env, n):
        gym.Wrapper.__init__(self, env)
        shape = env.observation_space.shape
        self.n = n
        self.frames = deque([], maxlen=n)
        self.observation_space = \
            spaces.Box(low=0, high=255, shape=((n,) + shape), dtype=env.observation_space.dtype)
    def reset(self):
        obs = self.env.reset()
        for _ in range(self.n):
            self.frames.append(obs)
        return self._get_obs()

    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        self.frames.append(obs)
        return self._get_obs(), reward, done, info

    def _get_obs(self):
        return np.array(self.frames)

In [ ]:
env = gym.make("BreakoutNoFrameskip-v4")
print(f"The original observation space is  {env.observation_space}")
env = ConcatObservations(env, 4)
print(f"The new observation space is  {env.observation_space}")


In [ ]:
env.reset()
obs, _, _, _  = env.step(env.action_space.sample())
print(f"Observation after taking a random step is  {obs.shape}")
env.close()

# Monitors for saving videos

In [ ]:
import gym
from gym.wrappers import Monitor
#env = gym.make('CartPole-v0')
env = Monitor(gym.make('CartPole-v0'), 'recording', force=True)
state = env.reset()
done = False

while not done:
    action = env.action_space.sample()
    state_next, reward, done, info = env.step(action)

env.close()


In [ ]:
# On Colab
from IPython.display import HTML
from base64 import b64encode
mp4 = open('recording/openaigym.video.3.59.video000000.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)


## Visualize Animation

In [ ]:
import gym
import matplotlib.pyplot as plt
import matplotlib.animation as animation

env = gym.make('Pong-v0')
obs = env.reset()
frames = [] # array to store state space at each step
for _ in range(300):
    frames.append(env.render(mode='rgb_array'))
    obs,reward,done, _ = env.step(env.action_space.sample())
    if done:
        break


In [ ]:
patch = plt.imshow(frames[0])
plt.axis('off')
def animate(i):
    patch.set_data(frames[i])
anim = animation.FuncAnimation(plt.gcf(), animate, \
                               frames=len(frames), interval=100)
#anim.save('random_agent.gif', writer='imagemagick')

![](https://github.com/Educat8n/Reinforcement-Learning-for-Game-Playing-and-More/blob/main/images/random_agent.gif?raw=1)

In [ ]:
# For Colab
from IPython.display import HTML
HTML(anim.to_html5_video())